# Vegetation Change Detection with Multi-Temporal Analysis

This notebook demonstrates how to detect and classify vegetation changes over time using satellite imagery from multiple temporal periods.

## Learning Objectives
- Understand change detection concepts and methodologies
- Create temporal composites from different periods
- Calculate delta indices (change between periods)
- Classify changes into meaningful categories
- Quantify and visualize vegetation change

## Prerequisites
- Completed 001_intro_ndvi.ipynb
- Google Earth Engine account
- Understanding of NDVI and spectral indices

## 1. Change Detection Concepts

### What is Change Detection?

Change detection is the process of identifying differences in the state of an object or phenomenon by observing it at different times.

### Approaches to Change Detection

1. **Image Differencing**: Subtract one image from another (e.g., dNDVI = NDVI_after - NDVI_before)
2. **Image Ratioing**: Ratio of two images
3. **Classification Comparison**: Compare classified images from different dates
4. **Time Series Analysis**: Analyze trends over many observations

### Our Approach: Delta Index Classification

We calculate the difference in spectral indices between periods and classify into 5 categories:

| Class | Description | dNDVI Range |
|-------|-------------|-------------|
| 1 | Strong Loss | < -0.15 |
| 2 | Moderate Loss | -0.15 to -0.05 |
| 3 | Stable | -0.05 to +0.05 |
| 4 | Moderate Gain | +0.05 to +0.15 |
| 5 | Strong Gain | > +0.15 |

## 2. Setup

In [ ]:
import ee
import folium
import matplotlib.pyplot as plt
import numpy as np

# Initialize Earth Engine
try:
    ee.Initialize()
    print("Earth Engine initialized successfully!")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Import platform modules
from engine.config import TEMPORAL_PERIODS, CHANGE_THRESHOLDS, CHANGE_CLASSES
from engine.composites import create_all_period_composites, create_fused_composite
from engine.indices import add_all_indices, calculate_delta_indices
from engine.change import (
    classify_change, 
    analyze_period_change,
    create_change_analysis,
    generate_change_statistics,
)

print("Platform modules imported!")
print(f"Available periods: {list(TEMPORAL_PERIODS.keys())}")

In [ ]:
# Define study area - Example: Area in Rondônia, Brazil (deforestation frontier)
rondonia_bbox = [-63.0, -10.5, -62.5, -10.0]

aoi = ee.Geometry.Rectangle(rondonia_bbox)

center_lat = (rondonia_bbox[1] + rondonia_bbox[3]) / 2
center_lon = (rondonia_bbox[0] + rondonia_bbox[2]) / 2

print(f"Study area: Rondônia, Brazil")
print(f"Center: {center_lat:.2f}°S, {center_lon:.2f}°W")

## 3. Understanding Temporal Periods

Our platform defines four temporal periods that span from 1985 to present.

In [ ]:
# Display period information
print("Temporal Periods Configuration")
print("=" * 60)
for name, info in TEMPORAL_PERIODS.items():
    print(f"\n{name}:")
    print(f"  Date range: {info['start']} to {info['end']}")
    print(f"  Sensors: {info['sensors']}")
    print(f"  Description: {info['description']}")

## 4. Create Temporal Composites

Create median composites for each period to compare.

In [ ]:
# Select periods to analyze
periods_to_analyze = ['1990s', 'present']

# Create composites for selected periods
composites = create_all_period_composites(
    aoi=aoi,
    periods=periods_to_analyze,
    cloud_threshold=20.0,
)

print(f"Created composites for: {list(composites.keys())}")

In [ ]:
# Add spectral indices to each composite
indices_to_calculate = ['ndvi', 'nbr']

for period_name in composites:
    composites[period_name] = add_all_indices(
        composites[period_name],
        indices=indices_to_calculate,
    )
    bands = composites[period_name].bandNames().getInfo()
    print(f"{period_name}: {bands}")

## 5. Visualize Before and After Composites

In [ ]:
# Visualization parameters
ndvi_vis = {
    'min': -0.2,
    'max': 0.8,
    'palette': ['#d73027', '#fc8d59', '#fee08b', '#d9ef8b', '#91cf60', '#1a9850']
}

# Create side-by-side comparison map
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add 1990s NDVI
ndvi_1990s = composites['1990s'].select('ndvi')
map_id_1990s = ndvi_1990s.getMapId(ndvi_vis)
folium.TileLayer(
    tiles=map_id_1990s['tile_fetcher'].url_format,
    attr='GEE',
    name='NDVI 1990s',
    overlay=True,
    show=True,
).add_to(m)

# Add present NDVI
ndvi_present = composites['present'].select('ndvi')
map_id_present = ndvi_present.getMapId(ndvi_vis)
folium.TileLayer(
    tiles=map_id_present['tile_fetcher'].url_format,
    attr='GEE',
    name='NDVI Present',
    overlay=True,
    show=False,
).add_to(m)

folium.LayerControl().add_to(m)
m

## 6. Calculate Delta Indices (Change)

In [ ]:
# Calculate change between periods manually
# dNDVI = NDVI_present - NDVI_1990s
# Positive values = vegetation gain
# Negative values = vegetation loss

dndvi_manual = ndvi_present.subtract(ndvi_1990s).rename('dndvi')

print("Calculated dNDVI (present - 1990s)")
print("Positive = vegetation gain, Negative = vegetation loss")

In [ ]:
# Using platform function
delta_indices = calculate_delta_indices(
    before=composites['1990s'],
    after=composites['present'],
    indices=indices_to_calculate,
)

print(f"Delta indices bands: {delta_indices.bandNames().getInfo()}")

## 7. Visualize Change (Delta NDVI)

In [ ]:
# Delta NDVI visualization - diverging color scheme
dndvi_vis = {
    'min': -0.4,
    'max': 0.4,
    'palette': [
        '#d7191c',  # Strong loss (red)
        '#fdae61',  # Moderate loss (orange)
        '#ffffbf',  # Stable (yellow)
        '#a6d96a',  # Moderate gain (light green)
        '#1a9641',  # Strong gain (dark green)
    ]
}

# Create change map
m_change = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add delta NDVI layer
dndvi = delta_indices.select('dndvi')
map_id_dndvi = dndvi.getMapId(dndvi_vis)
folium.TileLayer(
    tiles=map_id_dndvi['tile_fetcher'].url_format,
    attr='GEE',
    name='dNDVI (1990s → Present)',
).add_to(m_change)

folium.LayerControl().add_to(m_change)
m_change

## 8. Classify Change Into Categories

In [ ]:
# Display change thresholds
print("Change Classification Thresholds")
print("=" * 40)
for index_name, thresholds in CHANGE_THRESHOLDS.items():
    print(f"\n{index_name}:")
    for key, value in thresholds.items():
        print(f"  {key}: {value}")

In [ ]:
# Classify change manually using thresholds
thresholds = CHANGE_THRESHOLDS['dndvi']

# Start with "stable" (class 3)
change_class = ee.Image(3)

# Apply classification rules
change_class = (
    change_class
    .where(dndvi.lt(thresholds['strong_loss']), 1)  # Strong loss
    .where(dndvi.gte(thresholds['strong_loss']).And(dndvi.lt(thresholds['moderate_loss'])), 2)  # Moderate loss
    .where(dndvi.gt(thresholds['moderate_gain']).And(dndvi.lte(thresholds['strong_gain'])), 4)  # Moderate gain
    .where(dndvi.gt(thresholds['strong_gain']), 5)  # Strong gain
).rename('change_class')

print("Change classified into 5 categories!")

In [ ]:
# Using platform function
change_result = analyze_period_change(
    before=composites['1990s'],
    after=composites['present'],
    index='ndvi',
)

print(f"Change result bands: {change_result.bandNames().getInfo()}")

## 9. Visualize Classification Results

In [ ]:
# Change class visualization
class_vis = {
    'min': 1,
    'max': 5,
    'palette': ['#d7191c', '#fdae61', '#ffffbf', '#a6d96a', '#1a9641']
}

# Display class info
print("Change Classes")
print("=" * 40)
for class_num, info in CHANGE_CLASSES.items():
    print(f"Class {class_num}: {info['label']} ({info['color']})")

In [ ]:
# Create classified change map
m_class = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add classified change layer
class_image = change_result.select('change_class')
map_id_class = class_image.getMapId(class_vis)
folium.TileLayer(
    tiles=map_id_class['tile_fetcher'].url_format,
    attr='GEE',
    name='Change Classification',
).add_to(m_class)

# Add legend as HTML
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background: white; padding: 10px; border-radius: 5px;">
    <b>Change Classes</b><br>
    <span style="background: #d7191c; padding: 2px 10px;"></span> Strong Loss<br>
    <span style="background: #fdae61; padding: 2px 10px;"></span> Moderate Loss<br>
    <span style="background: #ffffbf; padding: 2px 10px;"></span> Stable<br>
    <span style="background: #a6d96a; padding: 2px 10px;"></span> Moderate Gain<br>
    <span style="background: #1a9641; padding: 2px 10px;"></span> Strong Gain
</div>
'''
m_class.get_root().html.add_child(folium.Element(legend_html))

folium.LayerControl().add_to(m_class)
m_class

## 10. Calculate Change Statistics

In [ ]:
# Generate statistics using platform function
stats = generate_change_statistics(
    change_image=change_result,
    aoi=aoi,
    scale=30,
)

print("Change Statistics")
print("=" * 50)
print(f"Total area analyzed: {stats.get('total_area_ha', 'N/A'):.1f} hectares")
print("\nArea by change class:")
for class_name, area in stats.get('area_by_class', {}).items():
    print(f"  {class_name}: {area:.1f} ha")

In [ ]:
# Calculate pixel counts per class
pixel_counts = class_image.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9,
).getInfo()

# Process histogram
histogram = pixel_counts['change_class']
class_names = ['', 'Strong Loss', 'Moderate Loss', 'Stable', 'Moderate Gain', 'Strong Gain']

print("\nPixel counts by class:")
for class_num, count in sorted(histogram.items()):
    class_idx = int(float(class_num))
    if 1 <= class_idx <= 5:
        print(f"  {class_names[class_idx]}: {count:,} pixels")

## 11. Create Change Pie Chart

In [ ]:
# Prepare data for pie chart
labels = []
sizes = []
colors = ['#d7191c', '#fdae61', '#ffffbf', '#a6d96a', '#1a9641']

for i in range(1, 6):
    key = str(i)
    if key in histogram:
        labels.append(class_names[i])
        sizes.append(histogram[key])

# Create pie chart
fig, ax = plt.subplots(figsize=(10, 8))
ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax.set_title(f'Vegetation Change Distribution (1990s → Present)\nStudy Area: Rondônia, Brazil')
plt.tight_layout()
plt.show()

## 12. Full Analysis Pipeline

Using the platform's high-level functions for a complete analysis.

In [ ]:
# Import the high-level analysis function
from services.change_orchestrator import analyze_vegetation_change

# Run complete analysis
results = analyze_vegetation_change(
    aoi=aoi,
    periods=['1990s', '2000s', '2010s', 'present'],
    indices=['ndvi', 'nbr'],
    reference_period='1990s',
)

print("Analysis complete!")
print(f"Composites created: {list(results['composites'].keys())}")
print(f"Change comparisons: {list(results['changes'].keys())}")

In [ ]:
# Display statistics summary
print("\nChange Statistics Summary")
print("=" * 60)
for comparison, stats in results['statistics'].items():
    print(f"\n{comparison}:")
    if stats:
        for key, value in stats.items():
            if isinstance(value, dict):
                print(f"  {key}:")
                for k, v in value.items():
                    print(f"    {k}: {v}")
            else:
                print(f"  {key}: {value}")

## Exercises

1. **Sequential Change Analysis**: Create change maps for each adjacent period pair (1990s→2000s, 2000s→2010s, 2010s→present) to see how deforestation has progressed over time.

2. **Different Indices**: Compare change detection results using NBR vs NDVI. Which index is more sensitive to deforestation?

3. **Custom Thresholds**: Experiment with different classification thresholds. How do results change with stricter or looser thresholds?

4. **Area Quantification**: Calculate the total hectares of forest lost between 1990s and present.

5. **Export Results**: Export the change classification map to Google Drive for use in GIS software.

## Summary

In this notebook, we learned:

1. **Change detection concepts**: Image differencing and delta indices
2. **Temporal composites**: Creating cloud-free composites from different periods
3. **Delta calculation**: Computing changes between periods (dNDVI, dNBR)
4. **Classification**: Converting continuous change values into discrete classes
5. **Statistics**: Quantifying change by area and pixel counts
6. **Visualization**: Creating effective change maps
7. **Full pipeline**: Using the platform for complete multi-period analysis

## Key Findings

The Rondônia study area shows significant vegetation loss between 1990s and present, consistent with known deforestation patterns in the Amazon arc of deforestation.

## Next Steps

- Explore the FastAPI endpoints for automated analysis
- Use the Streamlit dashboard for interactive exploration
- Apply machine learning for more sophisticated change detection